In [7]:
import pandas as pd
import numpy as np

In [2]:
#load data
icu_data = pd.read_csv('icu/icustays_cleaned.csv')
diagnosis = pd.read_csv('ed/diagnosis_cleaned.csv')
triage = pd.read_csv('ed/triage_cleaned.csv')
vitals = pd.read_csv('ed/vitals_cleaned.csv')

# Combine the icd_version and icd_code columns in diagnosis table
diagnosis['icd_combined'] = diagnosis['icd_version'].astype(str) + '-' + diagnosis['icd_code'].astype(str)
print(diagnosis.head())

unique_icd_combined_count = diagnosis['icd_combined'].nunique()
print('Number of unique icd_combined:', unique_icd_combined_count)

   subject_id   stay_id  seq_num icd_code  icd_version  \
0    10000032  32952584        1     4589            9   
1    10000032  32952584        2    07070            9   
2    10000032  32952584        3      V08            9   
3    10000032  39399961        1    78097            9   
4    10000032  39399961        2    34830            9   

                                           icd_title icd_combined  
0                                    HYPOTENSION NOS       9-4589  
1  UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...      9-07070  
2                         ASYMPTOMATIC HIV INFECTION        9-V08  
3                             ALTERED MENTAL STATUS       9-78097  
4                        ENCEPHALOPATHY, UNSPECIFIED      9-34830  
Number of unique icd_combined: 3894


In [ ]:
# this is just exploratory. we can decide whether to keep this or not later

cols = ['heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']

def range(df, columns):
    ranges = {}
    for col in cols:
        col_data = df[col].replace(0.0, np.nan) # skip values of 0
        ranges[col] = {
            'min': col_data.min(),
            'max': df[col].max()
        }
    return ranges

vitals_ranges = range(vitals, cols)
print("Ranges in vitals_cleaned:", vitals_ranges)

# Calculate ranges for triage_cleaned
triage_ranges = range(triage, cols)
print("Ranges in triage_cleaned:", triage_ranges)

Ranges in vitals_cleaned: {'heartrate': {'min': np.float64(1.0), 'max': np.float64(705.0)}, 'resprate': {'min': np.float64(0.34), 'max': np.float64(78.0)}, 'o2sat': {'min': np.float64(1.0), 'max': np.float64(972.0)}, 'sbp': {'min': np.float64(2.0), 'max': np.float64(854.0)}, 'dbp': {'min': np.float64(2.0), 'max': np.float64(97100.0)}}
Ranges in triage_cleaned: {'heartrate': {'min': np.float64(14.0), 'max': np.float64(1228.0)}, 'resprate': {'min': np.float64(1.0), 'max': np.float64(189.0)}, 'o2sat': {'min': np.float64(2.0), 'max': np.float64(1004.0)}, 'sbp': {'min': np.float64(1.0), 'max': np.float64(9656.0)}, 'dbp': {'min': np.float64(4.0), 'max': np.float64(9102.0)}}
